# NLQ to SQL Query Translator: Evaluation

This notebook contains our code to evaluate our Natural Language Query (NLQ) to Structured Query Language (SQL) generator. It requires functions from the nlq2sql and sqlExtract classes. It also assumes spaCy is already installed as spaCy comes pre-installed in the Google Colab environment.

Note: change file paths wherever appropriate to avoid errors.

# General Setup

We mount this notebook so that it can access Google Drive.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


We install dependencies and load the WikiSQL dataset.

In [2]:
!pip install datasets

In [3]:
from tqdm import tqdm
from datasets import load_dataset

In [4]:
dataset = load_dataset('wikisql')

Generating test split:   0%|          | 0/15878 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8421 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/56355 [00:00<?, ? examples/s]

The cell below describes the format of the WikiSQL dataset. Each row has an NLQ, the corresponding SQL query, and a set of attributes (column names of the table).

In [5]:
question = dataset ['train'][0]['question']
sqlQuery = dataset ['train'][0]['sql']['human_readable']
columns = dataset ['train'][0]['table']['header']

print("NLQ:", question)
print("SQL Query:", sqlQuery)
print("Columns/Attributes:", columns)

print('Train length:', len(dataset['train']))
print('Validation length:', len(dataset['validation']))
print('Test length:', len(dataset['test']))

NLQ: Tell me what the notes are for South Australia 
SQL Query: SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA
Columns/Attributes: ['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes']
Train length: 56355
Validation length: 8421
Test length: 15878


We import the nlq2sql class for evaluation and the sqlExtract class functions to extract attributes and values from SQL queries during evaluation.

In [6]:
%load_ext autoreload
%autoreload 2

from nlq2sql import *
from sqlExtract import *

The cell below shows an example usage of the sqlExtract's fetch where and select attributes functions.

In [7]:
sqlQuery = "SELECT COUNT Date,temp FROM table WHERE Location = davos AND Time > 45.7"
sqlExtractor = sqlExtract(sqlQuery)
whereAttr, whereAttrValue = sqlExtractor.fetch_where_attr()
print("Where Attributes:", whereAttr)
print("Where Attributes w/ Corresponding Values:", whereAttrValue)
print("Select Attributes:", sqlExtractor.fetch_select_attr())

Where Attributes: ['Location', 'Time']
Where Attributes w/ Corresponding Values: {'Location': 'davos', 'Time': '45.7'}
Select Attributes: {'Date': 'COUNT', 'temp': ''}


# Filter Test Data for Evaluation

We define filter functions to be those functions that help us filter our dataset to evaluate our model only on a specific subset of queries at a time. These functions also helped us analyze the intricacies of our model to understand its strengths and weaknesses. They also helped us develop our model iteratively so that it can gradually increase its complexity. Each filter function's condition is defined in the function's docstring.

In [8]:
def is_select_not_present_in_nlq(selectAttr, question):
  """Return true if the select attribute is not explicitly present in the NLQ."""
  # assume all the select attributes are explicitly present in the nlq
  for attr in selectAttr.keys():
    idx = question.lower().find(attr.lower())
    if attr != '*' and idx == -1:
        return True
    endIdx = idx + len(attr.lower()) - 1
    # if part of the select attr was found in the query, the character after
    # it (if exists) would not be a whitespace but would be an alphanum char
    if endIdx + 1 < len(question) and question[endIdx + 1].isalnum():
      return True
  return False

In [9]:
selectAttr = {'Start': ''}
question = "Where was the starting position when there were 182 laps?"
print(is_select_not_present_in_nlq(selectAttr, question))

True


In [10]:
def is_invalid_num_select_attr(selectAttr):
  """Return true if more than one attribute has to be selected."""
  return len(selectAttr) > 1

In [11]:
def is_invalid_constraint(sqlQuery):
  """Return true if an operator other than the = is used."""
  for char in ['<', '>', '>=', '<=', '!=', '<>']:
    if char in sqlQuery:
      return True
  return False

In [12]:
def is_invalid_num_where_attr(whereAttr):
  """Return true if more than one attribute has a constraint."""
  return len(whereAttr) > 1

In [13]:
def spaces_in_attr(iterable):
  """Return true if there is a space in the given set of attributes."""
  for attr in iterable:
    if ' ' in attr:
      return True
  return False

In [14]:
def attr_not_present_in_nlq(whereAttrValue, question):
  """Return true if the given set of attributes are not explicitly present in the NLQ."""
  for attr, value in whereAttrValue.items():
    idx = question.lower().find(attr.lower())
    if idx == -1:
      return True
    endIdx = idx + len(attr.lower()) - 1
    if endIdx + 1 < len(question) and question[endIdx + 1].isalnum():
      return True
  return False

The is_aggregate filter function filters out queries with aggregate function keywords in SQL. These keywords are always in uppercase in the SQL queries of the WikiSQL dataset due to which the function assumes the same.

In [15]:
def is_aggregate(sqlQuery):
  """Return true if an aggregate keyword exists in the NLQ."""
  aggr = ['MAX', 'MIN', 'SUM', 'COUNT', 'AVG']
  for agg in aggr:
    if agg in sqlQuery.split():
      return True
  return False

The filter function below checks if a value is not explicitly present in the NLQ. This is only true for a very few poorly formatted queries (about 20 out of 15,000 in the test set). We apply this filter by default to the test split of WikiSQL.

In [16]:
def value_not_present_in_nlq(whereAttrValue, question):
  """Return true if the values are not explicity present in the NLQ."""
  for attr, value in whereAttrValue.items():
      if question.lower().find(value.lower()) == -1:
        return True
  return False

Some NLQs had a thin space instead of a white space in the WikiSQL dataset which negatively affected the accuracy of our named-entity recognition (NER) model, so we are replacing them in the function below.

In [17]:
def remove_unicode_characters(text):
  # replace thin space with normal space
  return text.replace("\u2009", " ")

In the cell below, we define a function apply our desired filters and extract a partition of the WikiSQL test split to evaluate our model on.

In [18]:
def filter_test_data(filters={}):
  validFilters = {'is_select_not_present_in_nlq',
                   'is_invalid_num_select_attr',
                   'is_invalid_constraint',
                   'is_invalid_num_where_attr',
                   'spaces_in_select_attr',
                   'spaces_in_where_attr',
                   'attr_not_present_in_nlq',
                   'is_aggregate'}
  for filter in filters:
    assert filter in validFilters
  tagTestSet= []
  i = 0
  for row in tqdm(dataset['test']):
    question = row['question']
    sqlQuery = row['sql']['human_readable']
    columns = row['table']['header']
    question = remove_unicode_characters(question)
    sqlQuery = remove_unicode_characters(sqlQuery)
    sqlExtractor = sqlExtract(sqlQuery)
    for i in range(len(columns)):
      columns[i] = remove_unicode_characters(columns[i])
    selectAttr = sqlExtractor.fetch_select_attr()
    if 'is_select_not_present_in_nlq' in filters and is_select_not_present_in_nlq(selectAttr, question):
      continue
    if 'is_invalid_num_select_attr' in filters and is_invalid_num_select_attr(selectAttr):
      continue
    if 'is_invalid_constraint' in filters and is_invalid_constraint(sqlQuery):
      continue
    if 'is_aggregate' in filters and is_aggregate(sqlQuery):
      continue
    if 'spaces_in_select_attr' in filters and spaces_in_attr(selectAttr):
      continue
    if 'WHERE' in sqlQuery:
      whereAttr, whereAttrValue = sqlExtractor.fetch_where_attr()
      if 'is_invalid_num_where_attr' in filters and is_invalid_num_where_attr(whereAttr):
        continue
      if 'attr_not_present_in_nlq' in filters and attr_not_present_in_nlq(whereAttrValue, question):
        continue
      if 'spaces_in_where_attr' in filters and spaces_in_attr(whereAttr):
        continue
      if value_not_present_in_nlq(whereAttrValue, question):
        continue
    tagTestSet.append({'question': question, 'sqlQuery': sqlQuery, 'attr': columns})
  print(len(tagTestSet))
  return tagTestSet

The obtain_aggregate_queries function obtains aggregate queries with some default filters applied: Values and SELECT/WHERE attributes are explicitly present in the NLQ.

In [19]:
def obtain_aggregate_queries():
  tagTestSet= []
  i = 0
  for row in tqdm(dataset['test']):
    question = row['question']
    sqlQuery = row['sql']['human_readable']
    columns = row['table']['header']
    question = remove_unicode_characters(question)
    sqlQuery = remove_unicode_characters(sqlQuery)
    sqlExtractor = sqlExtract(sqlQuery)
    addedIndicator = False
    for i in range(len(columns)):
      columns[i] = remove_unicode_characters(columns[i])
    selectAttr = sqlExtractor.fetch_select_attr()
    if is_select_not_present_in_nlq(selectAttr, question):
      continue
    if 'WHERE' in sqlQuery:
      whereAttr, whereAttrValue = sqlExtractor.fetch_where_attr()
      if attr_not_present_in_nlq(whereAttrValue, question):
        continue
      if value_not_present_in_nlq(whereAttrValue, question):
        continue
    if is_aggregate(sqlQuery):
      tagTestSet.append({'question': question, 'sqlQuery': sqlQuery, 'attr': columns})
  print(len(tagTestSet))
  return tagTestSet

The obtain_absent_where_attr_queries function obtains queries whose where attributes are not explicitly present in the NLQ with some default filters applied: Values and SELECT attributes are explicitly present in the NLQ.

In [32]:
def obtain_absent_where_attr_queries():
  """Obtain queries where WHERE attr isn't present in the NLQ."""
  tagTestSet= []
  i = 0
  for row in tqdm(dataset['test']):
    question = row['question']
    sqlQuery = row['sql']['human_readable']
    columns = row['table']['header']
    question = remove_unicode_characters(question)
    sqlQuery = remove_unicode_characters(sqlQuery)
    sqlExtractor = sqlExtract(sqlQuery)
    for i in range(len(columns)):
      columns[i] = remove_unicode_characters(columns[i])
    selectAttr = sqlExtractor.fetch_select_attr()
    if is_select_not_present_in_nlq(selectAttr, question):
      continue
    if 'WHERE' in sqlQuery:
      whereAttr, whereAttrValue = sqlExtractor.fetch_where_attr()
      if value_not_present_in_nlq(whereAttrValue, question):
        continue
      if attr_not_present_in_nlq(whereAttrValue, question):
        tagTestSet.append({'question': question, 'sqlQuery': sqlQuery, 'attr': columns})
  print(len(tagTestSet))
  return tagTestSet

# Setup Evaluation Helper Functions

We install the large spaCy English model to perform tokenization in nlq2sql and the transformers library to perform custom NER.

In [21]:
import spacy
!python3 -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
!pip install spacy-transformers

In [23]:
def ignore_case_and_order(iterable):
  """Return the iterable in lower case as a set."""
  temp = []
  for item in iterable:
    if item is None:
      print(iterable)
    temp.append(item.lower())
  return set(temp)

In [24]:
def ignore_case(dictObj):
  """Return a dict with lower case keys and values."""
  temp = {}
  for key, value in dictObj.items():
    temp[key.lower()] = value.lower()
  return temp

In [25]:
def remove_quotes(sqlQuery):
  """Remove quotes from an SQL query."""
  return sqlQuery.replace('"', '')

The evaluate_data function defined below calculates accuracy on the filtered tagTestSet and prints statistics. It prints the SELECT accuracy, which is calculated as:
$$\frac{\text{#queries with correct SELECT attributes}}{\text{#total queries}}$$
the WHERE accuracy, which is calculated as:
$$\frac{\text{#queries with correct WHERE attributes}}{\text{#total queries}}$$
the Value accuracy, which is calculated as:
$$\frac{\text{#queries with correct Values}}{\text{#total queries}}$$
the Overall accuracy, which is calculated as:
$$\frac{\text{#correct queries}}{\text{#total queries}}$$
Correct queries are those queries with correct SELECT attributes and correct WHERE attributes associated with their correct corresponding values and operators.

It prints a "1" if the SELECT/WHERE attributes and values in the predicted and expected SQL query match. Otherwise, it prints the NLQ, attributes, predicted SQL query, and the expected SQL query. Along with this information, it also prints a progress fraction: $$\frac{\text{#queries completed}}{\text{#total queries}}$$

In [26]:
import spacy_transformers

def evaluate_data(tagTestSet, nlp_ner, rule, nlp_aggr=None):
  """Calculate accuracy on test data and print statistics."""
  numCorrectOverall = 0
  numCorrectSelect = 0
  numCorrectWhere = 0
  numCorrectValues = 0
  nlp = spacy.load("en_core_web_lg")
  i = 0
  for obj in tagTestSet:
      sqlQuery = remove_quotes(obj['sqlQuery'])
      attributes = obj['attr']
      nlq = obj['question']
      nQueryObj = nlq2sql(nlq, "table", attributes, nlp, nlp_ner, rule)
      isSelect = False
      pred = nQueryObj.get_sql_query()
      sqlExtractor = sqlExtract(sqlQuery)
      correctSelectAttr = sqlExtractor.fetch_select_attr()
      if ignore_case(nQueryObj.selectAttributes) == ignore_case(correctSelectAttr):
          numCorrectSelect += 1
          isSelect = True
      correctWhereAttr, whereAttrVals = sqlExtractor.fetch_where_attr()
      if ignore_case_and_order(nQueryObj.whereAttributes.keys()) == ignore_case_and_order(correctWhereAttr):
          numCorrectWhere += 1
      if ignore_case_and_order(nQueryObj.whereAttributes.values()) == ignore_case_and_order(whereAttrVals.values()):
          numCorrectValues += 1
      if isSelect and ignore_case(nQueryObj.whereAttributes) == ignore_case(whereAttrVals):
          numCorrectOverall += 1
          print(f"{i} / {len(tagTestSet)}", 1)
      else:
        print(f"{i} / {len(tagTestSet)}", obj['question'], obj['attr'], pred, sqlQuery)
      i += 1
  print('Select accuracy', (numCorrectSelect / len(tagTestSet)) * 100)
  print('Where accuracy', (numCorrectWhere / len(tagTestSet)) * 100)
  print('Value accuracy', (numCorrectValues / len(tagTestSet)) * 100)
  print('Overall accuracy', (numCorrectOverall / len(tagTestSet)) * 100)

# Evaluation

This section consists of our results from evaluating our NLP to SQL Query translator on various cases which represent specific subsets of queries from the WikiSQL dataset. We have analyzed these results in detail in our final report.

We load 2 models: one that supports tagging SELECT attributes with their corresponding aggregate functions and one that doesn't. The nlp_ner model doesn't support aggregation, that is, it only tags words as "SELECT" or "CONSTRAINT". The nlp_aggr model supports aggregation, that is, it tags words as "COUNT SELECT", "MAX SELECT", "MIN SELECT", "AVG SELECT", "SELECT", and "CONSTRAINT".

In [27]:
nlp_ner = spacy.load("/content/drive/MyDrive/train_spacy/model-best-no-aggr")

In [28]:
nlp_aggr = spacy.load("/content/drive/MyDrive/train_spacy/model-best")

# Case 1: No Aggregation, Attributes Explicitly Present in NLQ, Rule 1

We evaluate accuracy assuming that

*   the SELECT and WHERE attributes are explicitly present in the NLQ
*   there are no aggregate functions like MAX, MIN, COUNT, or others
*   nlq2sql uses rule 1 (defined in our report)

We also assume that there are no WHERE clauses with any operator other than '=' but no such queries exist when we also impose the condition that attributes need to be explicitly present in the NLQ.

The ner model used in the evaluation below doesn't support aggregation.

In [29]:
filters = {'is_select_not_present_in_nlq',
           'is_invalid_constraint',
           'attr_not_present_in_nlq',
           'is_aggregate'}
tagTestSet = filter_test_data(filters)
print(len(tagTestSet))

100%|██████████| 15878/15878 [00:06<00:00, 2383.87it/s]

4855
4855


In [33]:
evaluate_data(tagTestSet, nlp_ner, 'rule1')

0 / 4855 1
1 / 4855 1
2 / 4855 1
3 / 4855 1
4 / 4855 1
5 / 4855 1
6 / 4855 1
7 / 4855 1
8 / 4855 1
9 / 4855 1
10 / 4855 1
11 / 4855 1
12 / 4855 1
13 / 4855 1
14 / 4855 If % lunsford is 51.82% what is the % mcconnell in Letcher? ['County', 'Precincts', 'Lunsford', '% Lunsford', 'McConnell', '% McConnell', 'Total'] SELECT % mcconnell FROM table WHERE lunsford = 51.82% SELECT % McConnell FROM table WHERE % Lunsford = 51.82%
15 / 4855 1
16 / 4855 what's the original air date with title  "hell" ['No. in series', 'No. in season', 'Title', 'Directed by', 'Written by', 'Original air date'] SELECT original air date FROM table WHERE title = "hell" SELECT Original air date FROM table WHERE Title = Hell
17 / 4855 1
18 / 4855 What is the percentage of the  trance- n himalaya zone that corresponds with the high hill zone is 25%? ['Particulars and Characteristics', 'Shivalik Zone', 'Mid-Hill Zone', 'High hill zone', 'Trance- n Himalaya Zone'] SELECT * FROM table WHERE high hill zone = 25% SELECT Tran

Our accuracy statistics for this case is as follows.

*   Select accuracy: 95.43%
*   Where accuracy: 85.81%
*   Value accuracy: 90.94%
*   Overall accuracy 80.86%


# Case 2: No Aggregation, Attributes Explicitly Present in NLQ, Rule 2

We evaluate accuracy with the same conditions as before, but nlq2sql uses rule 2 instead (defined in our report).

In [34]:
evaluate_data(tagTestSet, nlp_ner, 'rule2')

0 / 4855 1
1 / 4855 1
2 / 4855 1
3 / 4855 1
4 / 4855 1
5 / 4855 1
6 / 4855 1
7 / 4855 1
8 / 4855 1
9 / 4855 1
10 / 4855 1
11 / 4855 1
12 / 4855 1
13 / 4855 1
14 / 4855 1
15 / 4855 1
16 / 4855 what's the original air date with title  "hell" ['No. in series', 'No. in season', 'Title', 'Directed by', 'Written by', 'Original air date'] SELECT original air date FROM table WHERE original air date = "hell" SELECT Original air date FROM table WHERE Title = Hell
17 / 4855 1
18 / 4855 What is the percentage of the  trance- n himalaya zone that corresponds with the high hill zone is 25%? ['Particulars and Characteristics', 'Shivalik Zone', 'Mid-Hill Zone', 'High hill zone', 'Trance- n Himalaya Zone'] SELECT * FROM table WHERE trance- n himalaya zone = trance- n himalaya AND high hill zone = 25% SELECT Trance- n Himalaya Zone FROM table WHERE High hill zone = 25%
19 / 4855 1
20 / 4855 1
21 / 4855 1
22 / 4855 1
23 / 4855 1
24 / 4855 Which song has picturization by only vijay? ['Track #', 'Song', 'S

Our accuracy statistics for this case is as follows.


*   Select accuracy: 95.43%
*   Where accuracy: 82.35%
*   Value accuracy: 90.26%
*   Overall accuracy 78.41%

# Case 4: No Aggregation, Attributes Explicitly Present in NLQ, Only One WHERE and SELECT Attribute

We evaluate accuracy assuming that

*   the SELECT and WHERE attributes are explicitly present in the NLQ
*   there are no aggregate functions like MAX, MIN, COUNT, or others
*   nlq2sql uses rule 1 (defined in our report)
*   only one SELECT and WHERE attribute exist

We also assume that there are no WHERE clauses with any operator other than '=' but no such queries exist when we also impose the condition that attributes need to be explicitly present in the NLQ.

The ner model we used in the evaluation below doesn't support aggregation.



In [30]:
filters = {'is_select_not_present_in_nlq',
           'is_invalid_constraint',
           'is_invalid_num_select_attr',
           'is_invalid_num_where_attr',
           'attr_not_present_in_nlq',
           'is_aggregate'}
tagTestSet = filter_test_data(filters)
print(len(tagTestSet))

100%|██████████| 15878/15878 [00:06<00:00, 2435.85it/s]

3932
3932


In [31]:
evaluate_data(tagTestSet, nlp_ner, 'rule1')

0 / 3932 1
1 / 3932 1
2 / 3932 1
3 / 3932 1
4 / 3932 1
5 / 3932 1
6 / 3932 1
7 / 3932 1
8 / 3932 1
9 / 3932 1
10 / 3932 1
11 / 3932 1
12 / 3932 1
13 / 3932 1
14 / 3932 1
15 / 3932 1
16 / 3932 what's the original air date with title  "hell" ['No. in series', 'No. in season', 'Title', 'Directed by', 'Written by', 'Original air date'] SELECT original air date FROM table WHERE title = "hell" SELECT Original air date FROM table WHERE Title = Hell
17 / 3932 1
18 / 3932 What is the percentage of the  trance- n himalaya zone that corresponds with the high hill zone is 25%? ['Particulars and Characteristics', 'Shivalik Zone', 'Mid-Hill Zone', 'High hill zone', 'Trance- n Himalaya Zone'] SELECT * FROM table WHERE high hill zone = 25% SELECT Trance- n Himalaya Zone FROM table WHERE High hill zone = 25%
19 / 3932 1
20 / 3932 1
21 / 3932 1
22 / 3932 1
23 / 3932 Which song has picturization by only vijay? ['Track #', 'Song', 'Singers', 'Picturization', 'Length', 'Lyricist'] SELECT song FROM table WH

Our accuracy statistics for this case is as follows.

*   Select accuracy: 95.37%
*   Where accuracy: 88.73%
*   Value accuracy: 93.54%
*   Overall accuracy 83.62%


#Case 5: Only Aggregate Function Queries, Attributes Explicitly Present in NLQ, Rule 3
We evaluate accuracy assuming that

*   the SELECT and WHERE attributes are explicitly present in the NLQ
*   there are ONLY aggregate functions like MAX, MIN, COUNT, or others.
*   nlq2sql uses rule 3 (defined in our report)

We also assume that there are no WHERE clauses with any operator other than '=' but no such queries exist when we also impose the condition that attributes need to be explicitly present in the NLQ.

The ner model we used in the evaluation below supports aggregation.

In [44]:
tagTestSet = obtain_aggregate_queries()
print(len(tagTestSet))

100%|██████████| 15878/15878 [00:04<00:00, 3592.94it/s]

2027
2027


In [48]:
evaluate_data(tagTestSet, nlp_aggr, 'rule3')

0 / 2027 what is the total number of total w–l where doubles w–l is 11–11 ['Player', 'Total W–L', 'Singles W–L', 'Doubles W–L', 'Ties played', 'Debut', 'Years played'] SELECT * FROM table WHERE doubles w–l = 11–11 SELECT COUNT Total W–L FROM table WHERE Doubles W–L = 11–11
1 / 2027 what is the total number of singles w–l where doubles w–l is 11–14 ['Player', 'Total W–L', 'Singles W–L', 'Doubles W–L', 'Ties played', 'Debut', 'Years played'] SELECT * FROM table WHERE doubles w–l = 11–14 SELECT COUNT Singles W–L FROM table WHERE Doubles W–L = 11–14
2 / 2027 1
3 / 2027 1
4 / 2027 What is the Frequency at the Market/Rank of Burlington - Plattsburgh , Vermont - New York /143? ['Calls', 'Frequency', 'Branding', 'Format', 'Market/Rank', 'Timeslot', 'Group owner'] SELECT Frequency FROM table WHERE market/rank = Burlington - Plattsburgh , Vermont - New York /143 SELECT COUNT Frequency FROM table WHERE Market/Rank = Burlington - Plattsburgh , Vermont - New York /143
5 / 2027 Of the years that had

Our accuracy statistics for this case are as follows.

*   Select accuracy: 51.26%
*   Where accuracy: 69.46%
*   Value accuracy: 74.54%
*   Overall accuracy 36.16%

#Case 6: Operator Accuracy Evaluation
We evaluate the accuracy of our rule-based determine operator function which identifies which operator ('<', '>', '=') goes with which attribute-value pair in the WHERE clauses. We evaluate accuracy on the entire WikiSQL dataset below. Note that we are only evaluating the accuracy of operators here assuming that the WHERE attributes have already been identified correctly so we don't use any of our pre-defined rules or our NER model. We calculate accuracy using the following formula:

$$\frac{\text{#queries with correctly predicted operators in the WHERE clause}}{\text{#total queries}}$$


Because of a quirk in the WikiSQL dataset, about 50 queries have WHERE values that contain operators ("=", "<" and/or ">") in them. Therefore, the filter_invalid_operator_queries function ensures that those queries are ignored.

In [41]:
def filter_invalid_operator_queries():
  """Remove queries with values that have operators in them from the dataset."""
  validQueries = []
  for row in dataset['test']:
    question = row['question']
    sqlQuery = row['sql']['human_readable']
    columns = row['table']['header']

    question = remove_unicode_characters(question)
    sqlQuery = remove_unicode_characters(sqlQuery)
    sqlQuery = remove_quotes(sqlQuery)

    sqlExtractor = sqlExtract(sqlQuery)
    whereAttr, whereAttrValue = sqlExtractor.fetch_where_attr()

    flag = False
    allOperators = ['=', '>', '<']

    for operator in allOperators:
      for attr, value in whereAttrValue.items():
        if operator in attr or operator in value:
          flag = True
    if flag:
      continue

    validQueries.append({'question': question, 'sqlQuery': sqlQuery, 'attr': columns})
  return validQueries

In [42]:
validQueries = filter_invalid_operator_queries()
numQueries = len(validQueries)
print(numQueries)

15734


In [43]:
numCorrect = 0
count = 0
nlp = spacy.load("en_core_web_lg")

for query in validQueries:
  nlq = query['question']
  sqlQuery = query['sqlQuery']
  attributes = query['attr']

  sqlExtractor = sqlExtract(sqlQuery)

  whereIdx = sqlQuery.find('WHERE')
  if whereIdx == -1:
    whereAttrValue = {}
  else:
    whereAttr, whereAttrValue = sqlExtractor.fetch_where_attr()

  trueCompOperators = sqlExtractor.fetch_comparison_operator()

  nQueryObj = nlq2sql(question, "table", columns, nlp, nlp_ner, "rule1")
  nQueryObj.determine_operator(whereAttrValue)
  predictedCompOperators = nQueryObj.comparisonOperators

  if trueCompOperators != predictedCompOperators:
    print(f"{count} / {numQueries} {sqlQuery} {predictedCompOperators}")
  else:
    print(f"{count} / {numQueries} 1")
    numCorrect += 1

  count += 1

print('Operator accuracy', (numCorrect / numQueries) * 100)

Streaming output truncated to the last 5000 lines.
10735 / 15734 SELECT MIN Long FROM table WHERE Player = santana moss AND Att. > 1 ['=', '=']
10736 / 15734 1
10737 / 15734 SELECT SUM Yards FROM table WHERE Player = jason campbell AND Long < 23 ['=', '=']
10738 / 15734 1
10739 / 15734 1
10740 / 15734 1
10741 / 15734 1
10742 / 15734 1
10743 / 15734 1
10744 / 15734 1
10745 / 15734 1
10746 / 15734 1
10747 / 15734 1
10748 / 15734 1
10749 / 15734 SELECT Title FROM table WHERE Rating > 9.3 AND 18-49 < 5.5 AND Airdate = january 11, 2007 ['=', '=', '=']
10750 / 15734 SELECT SUM Viewers FROM table WHERE Rating > 7.8 AND Airdate = october 26, 2006 AND Share > 14 ['=', '=', '=']
10751 / 15734 SELECT MIN Viewers FROM table WHERE Episode = 2 AND 18-49 > 4.9 ['=', '=']
10752 / 15734 1
10753 / 15734 SELECT Mintage FROM table WHERE Artist = royal canadian mint engravers AND Year < 2008 AND Issue Price = $102.95 ['=', '=', '=']
10754 / 15734 SELECT Theme FROM table WHERE Year < 2006 AND Issue Price = 

Our accuracy statistics for this case are as follows.

*   Overall accuracy 81.42%

# Case 8: WHERE Attribute Classification Accuracy

We evaluate the accuracy of our word-embedding-based approach to handle NLQs where the attribute associated with an identified value in the expected WHERE clause isn't explicitly mentioned in the NLQ. In this approach, we find the attribute that has the greatest cosine similarity with the identified value and associate that attribute with the identified value. We calculate accuracy using the following formula:

$$\frac{\text{#values for which attributes were correctly predicted}}{\text{#total values evaluated}}$$

Note that we assume the values have been correctly identified in this section so we don't use our nlp_ner model to identify constraints even though we pass the model to the nlq2sql class.

We only count a value to be evaluated if predicting an attribute was possible for that value. Sometimes, predicting an attribute is impossible because there is no word embedding for a token in the value or attribute.

We also calculate the number of queries for which at least one value didn't have an attribute predicted as a percentage of the total number of queries.

In [33]:
tagTestSet = obtain_absent_where_attr_queries()
print(len(tagTestSet))

100%|██████████| 15878/15878 [00:06<00:00, 2514.66it/s]

4655
4655


We download [GloVe](https://nlp.stanford.edu/projects/glove/) word embeddings.

In [49]:
import gensim.downloader
embed = gensim.downloader.load("glove-wiki-gigaword-200")

[==================================================] 100.0% 252.1/252.1MB downloaded


In [59]:
numValuesEvaluated = 0
numCorrect = 0
unevaluatedValueQueries = 0
nlp = spacy.load("en_core_web_lg")
i = 0
for obj in tagTestSet:
  i += 1
  sqlQuery = remove_quotes(obj['sqlQuery'])
  attributes = obj['attr']
  nlq = obj['question']
  nQueryObj = nlq2sql(nlq, "table", attributes, nlp, nlp_ner)
  sqlExtractor = sqlExtract(sqlQuery)
  whereAttr, whereAttrValue = sqlExtractor.fetch_where_attr()
  unevaluatedIndicator = False
  for attr, value in whereAttrValue.items():
    mostSimilarAttr = nQueryObj.get_most_similar_attribute(value, embed)
    if mostSimilarAttr is not None:
      numValuesEvaluated += 1
      if mostSimilarAttr.lower() == attr.lower():
        numCorrect += 1
        print(f"{i} / {len(tagTestSet)} 1")
      else:
        print(f"{i} / {len(tagTestSet)} Value: {value} Pred: {mostSimilarAttr.lower()} Correct: {attr.lower()}")
    else:
      print(f"{i} / {len(tagTestSet)} Value: {value} Pred: {mostSimilarAttr} Correct: {attr.lower()}")
      unevaluatedIndicator = True
  if unevaluatedIndicator:
    unevaluatedValueQueries += 1
if numValuesEvaluated > 0:
  print(f"Attribute Classification Accuracy: {(numCorrect / numValuesEvaluated) * 100}")
  print(f"% of queries where at least 1 value was not evaluated: {((len(tagTestSet) - unevaluatedValueQueries)/ len(tagTestSet)) * 100}")
else:
  print("0 queries evaluated.")

Streaming output truncated to the last 5000 lines.
1504 / 4655 Value: milwaukee county stadium Pred: attendance Correct: game site
1505 / 4655 Value: ontario Pred: shot pct. Correct: locale
1506 / 4655 Value: 77 Pred: ends lost Correct: shot pct.
1506 / 4655 Value: 11 Pred: shot pct. Correct: blank ends
1506 / 4655 Value: 7 Pred: shot pct. Correct: stolen ends
1506 / 4655 Value: 44 Pred: ends lost Correct: ends won
1507 / 4655 Value: the netherlands Pred: market value ( usd million) Correct: headquarters
1508 / 4655 1
1508 / 4655 Value: 23 Pred: seasons Correct: 3rd places
1509 / 4655 1
1509 / 4655 1
1510 / 4655 Value: dodge Pred: driver Correct: make
1511 / 4655 1
1512 / 4655 1
1513 / 4655 1
1513 / 4655 1
1514 / 4655 1
1514 / 4655 Value: 17 Pred: season Correct: podiums
1514 / 4655 Value: 0 Pred: wins Correct: fastest laps
1515 / 4655 1
1516 / 4655 1
1517 / 4655 Value: 6 Pred: year Correct: rank
1517 / 4655 Value: 617 / 188 Pred: year Correct: height feet / m
1518 / 4655 1
1518 / 4655

Our accuracy statistics for this case are as follows.

*   Attribute Classification Accuracy: 38.91%
*   Percentage of queries for which the attribute for at least 1 value could not be predicted: 83.37%



# View Queries for which SELECT attributes are not explicitly present in the NLQ

In this section, we display what kind of NLQs don't have the SELECT attributes explicitly present in them so that we can come up with future ways to address these types of queries.

In [68]:
def obtain_absent_select_queries():
  """Return the queries where the SELECT attributes aren't explicitly present in the NLQ."""
  tagTestSet= []
  i = 0
  for row in tqdm(dataset['test']):
    question = row['question']
    sqlQuery = row['sql']['human_readable']
    columns = row['table']['header']
    question = remove_unicode_characters(question)
    sqlQuery = remove_unicode_characters(sqlQuery)
    sqlExtractor = sqlExtract(sqlQuery)
    addedIndicator = False
    for i in range(len(columns)):
      columns[i] = remove_unicode_characters(columns[i])
    selectAttr = sqlExtractor.fetch_select_attr()
    if is_select_not_present_in_nlq(selectAttr, question):
      tagTestSet.append({'question': question, 'sqlQuery': sqlQuery, 'attr': columns})
  print(f"\n% of NLQs where the SELECT attr is not explicitly present in the NLQ: {(len(tagTestSet)/ len(dataset['test'])) * 100}\n")
  return tagTestSet

In [69]:
obtain_absent_select_queries()

100%|██████████| 15878/15878 [00:06<00:00, 2639.12it/s]



% of NLQs where the SELECT attr is not explicitly present in the NLQ: 24.259982365537223



[{'question': 'What clu was in toronto 1995-96',
  'sqlQuery': 'SELECT School/Club Team FROM table WHERE Years in Toronto = 1995-96',
  'attr': ['Player',
   'No.',
   'Nationality',
   'Position',
   'Years in Toronto',
   'School/Club Team']},
 {'question': 'which club was in toronto 2003-06',
  'sqlQuery': 'SELECT School/Club Team FROM table WHERE Years in Toronto = 2003-06',
  'attr': ['Player',
   'No.',
   'Nationality',
   'Position',
   'Years in Toronto',
   'School/Club Team']},
 {'question': 'how many schools or teams had jalen rose',
  'sqlQuery': 'SELECT COUNT School/Club Team FROM table WHERE Player = Jalen Rose',
  'attr': ['Player',
   'No.',
   'Nationality',
   'Position',
   'Years in Toronto',
   'School/Club Team']},
 {'question': 'Where was Assen held?',
  'sqlQuery': 'SELECT Round FROM table WHERE Circuit = Assen',
  'attr': ['No',
   'Date',
   'Round',
   'Circuit',
   'Pole Position',
   'Fastest Lap',
   'Race winner',
   'Report']},
 {'question': 'What was t

We found that about 24% of the queries in the test split do not have the SELECT attributes explicitly present in them. We have discussed potential ways to address these types of queries in our report's Future Work section.